In [1]:
#As always, lets start by importing the libraries we need
import json    
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable

#And the tools from Sklearn to do our clustering
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.metrics import mean_squared_error

from scipy.sparse import csr_matrix

from datetime import datetime


#To make things cleaner, lets also not display all the warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
businesses = []
with open('business.json') as f:
    for line in f:
        businesses.append(json.loads(line))
        
reviews = []
with open('review.json') as f:
    for line in f:
        reviews.append(json.loads(line))
        
testBusiness = businesses[1]
testReview = reviews[1]

print(testBusiness)
print('')
print(testReview)


{'business_id': 'QXAEGFB4oINsVuTFxEYKFQ', 'name': 'Emerald Chinese Restaurant', 'address': '30 Eglinton Avenue W', 'city': 'Mississauga', 'state': 'ON', 'postal_code': 'L5R 3E7', 'latitude': 43.6054989743, 'longitude': -79.652288909, 'stars': 2.5, 'review_count': 128, 'is_open': 1, 'attributes': {'RestaurantsReservations': 'True', 'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': True, 'brunch': False, 'breakfast': False}", 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'Caters': 'True', 'NoiseLevel': "u'loud'", 'RestaurantsTableService': 'True', 'RestaurantsTakeOut': 'True', 'RestaurantsPriceRange2': '2', 'OutdoorSeating': 'False', 'BikeParking': 'False', 'Ambience': "{'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': True}", 'HasTV': 'False', 'WiFi': "u'no'", 'GoodForKids': 'True', 'Alcohol': "u

In [3]:
#Put all data into dataframes 

review_id = []
user_id = []
business_id = []
stars = []
date = []

for review in reviews:
    review_id.append(review['review_id'])
    user_id.append(review['user_id'])
    business_id.append(review['business_id'])
    stars.append(review['stars'])
    date.append(review['date'])

ratingsDF = pd.DataFrame({'review_id': review_id,
                             'user_id': user_id,
                             'business_id': business_id,
                         'stars':stars,
                         'date':date})
ratingsDF

,review_id,user_id,business_id,stars,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,2017-01-14 21:30:33
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,2016-11-09 20:09:03
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,2018-01-09 20:56:38
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,2018-01-30 23:07:38
...,...,...,...,...,...
6685895,0pCaq1hqAViS2N6ldlp_sQ,Xu_YazIajfsc7SRj1wJ6Mg,RXBFk3tVBxiTf3uOt9KExQ,5.0,2018-07-03 12:17:27
6685896,jPCXuSBbI_cPocgoNBhdkg,V6BjjQICDO4q7TT3ZhaPWw,yA6dKNm_zl1ucZCnwW8ZCg,1.0,2013-05-22 15:32:08
6685897,zwzO3yPdGbdgGy3-XQ0SWg,PFiIECX8wuvi7P-1mCvIjA,a192hdM0_UVCYLwPJv1Qwg,5.0,2018-11-13 21:15:20
6685898,o8Nc2BJhKJXM6tRHJDRuPA,wu5R5_N7q5iqCBh6NBX26Q,kOo4ZY2UQAX4j312mzQ8mA,5.0,2018-08-13 18:09:16


In [4]:
business_id = []
business_name = []
business_city = []
business_state = []
business_catogories = []

for business in businesses:
    business_id.append(business['business_id'])
    business_name.append(business['name'])
    business_city.append(business['city'])
    business_state.append(business['state'])    
    
    categories = None
    if business['categories'] != None:
        categories = ""
        for category in business['categories']:
            categories += category
            
    business_catogories.append(categories)
    
businessDF = pd.DataFrame({'business_id': business_id,
                           'name': business_name,
                             'city': business_city,
                             'state': business_state, 
                          'categories': business_catogories})
businessDF

,business_id,name,city,state,categories
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,Phoenix,AZ,"Golf, Active Life"
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,Mississauga,ON,"Specialty Food, Restaurants, Dim Sum, Imported..."
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,Charlotte,NC,"Sushi Bars, Restaurants, Japanese"
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,Goodyear,AZ,"Insurance, Financial Services"
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,Charlotte,NC,"Plumbing, Shopping, Local Services, Home Servi..."
...,...,...,...,...,...
192604,nqb4kWcOwp8bFxzfvaDpZQ,Sanderson Plumbing,North Las Vegas,NV,"Water Purification Services, Water Heater Inst..."
192605,vY2nLU5K20Pee-FdG0br1g,Chapters,Newmarket,ON,"Books, Mags, Music & Video, Shopping"
192606,MiEyUDKTjeci5TMfxVZPpg,Phoenix Pavers,Phoenix,AZ,"Home Services, Contractors, Landscaping, Mason..."
192607,zNMupayB2jEHVDOji8sxoQ,Beasley's Barber Shop,Mesa,AZ,"Beauty & Spas, Barbers"


In [5]:
business_id = []
business_name = []
business_city = []
business_state = []
business_catogory_1 = []
business_catogory_2 = []
business_catogory_3 = []

for business in businesses:
    business_id.append(business['business_id'])
    business_name.append(business['name'])
    business_city.append(business['city'])
    business_state.append(business['state'])    
    
    categories = None
    
    if business['categories'] != None:
        categories = business['categories'].split(',')

        try:
            business_catogory_1.append(categories[0].strip())
        except:
            business_catogory_1.append(None)

        try:
            business_catogory_2.append(categories[1].strip())
        except:
            business_catogory_2.append(None)
            
        try:
            business_catogory_3.append(categories[2].strip())
        except:
            business_catogory_3.append(None)
    else:
        business_catogory_1.append(None)
        business_catogory_2.append(None)
        business_catogory_3.append(None)

In [6]:
businessDF = pd.DataFrame({'business_id': business_id,
                           'name': business_name,
                             'city': business_city,
                             'state': business_state, 
                          'category1': business_catogory_1, 
                          'category2': business_catogory_2,
                          'category3': business_catogory_3})
businessDF

,business_id,name,city,state,category1,category2,category3
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,Phoenix,AZ,Golf,Active Life,None
1,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,Mississauga,ON,Specialty Food,Restaurants,Dim Sum
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,Charlotte,NC,Sushi Bars,Restaurants,Japanese
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,Goodyear,AZ,Insurance,Financial Services,None
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,Charlotte,NC,Plumbing,Shopping,Local Services
...,...,...,...,...,...,...,...
192604,nqb4kWcOwp8bFxzfvaDpZQ,Sanderson Plumbing,North Las Vegas,NV,Water Purification Services,Water Heater Installation/Repair,Home Services
192605,vY2nLU5K20Pee-FdG0br1g,Chapters,Newmarket,ON,Books,Mags,Music & Video
192606,MiEyUDKTjeci5TMfxVZPpg,Phoenix Pavers,Phoenix,AZ,Home Services,Contractors,Landscaping
192607,zNMupayB2jEHVDOji8sxoQ,Beasley's Barber Shop,Mesa,AZ,Beauty & Spas,Barbers,None


In [14]:
ourStates = set(businessDF['state'])
ourStates

{'AB',
 'AK',
 'AL',
 'AR',
 'AZ',
 'BAS',
 'BC',
 'CA',
 'CON',
 'CT',
 'DOW',
 'DUR',
 'FL',
 'GA',
 'IL',
 'NC',
 'NE',
 'NJ',
 'NM',
 'NV',
 'NY',
 'OH',
 'ON',
 'PA',
 'QC',
 'SC',
 'TN',
 'TX',
 'UT',
 'VA',
 'VT',
 'WA',
 'WI',
 'XGL',
 'XGM',
 'XWY'}

In [20]:
import plotly.graph_objects as go

# Load data frame and tidy it.
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_us_ag_exports.csv')
correctStates = set(df['code'])
df

,code,state,category,total exports,beef,pork,poultry,dairy,fruits fresh,fruits proc,total fruits,veggies fresh,veggies proc,total veggies,corn,wheat,cotton
0,AL,Alabama,state,1390.63,34.4,10.6,481.0,4.06,8.0,17.1,25.11,5.5,8.9,14.33,34.9,70.0,317.61
1,AK,Alaska,state,13.31,0.2,0.1,0.0,0.19,0.0,0.0,0.00,0.6,1.0,1.56,0.0,0.0,0.00
2,AZ,Arizona,state,1463.17,71.3,17.9,0.0,105.48,19.3,41.0,60.27,147.5,239.4,386.91,7.3,48.7,423.95
3,AR,Arkansas,state,3586.02,53.2,29.4,562.9,3.53,2.2,4.7,6.88,4.4,7.1,11.45,69.5,114.5,665.44
4,CA,California,state,16472.88,228.7,11.1,225.4,929.95,2791.8,5944.6,8736.40,803.2,1303.5,2106.79,34.6,249.3,1064.95
5,CO,Colorado,state,1851.33,261.4,66.0,14.0,71.94,5.7,12.2,17.99,45.1,73.2,118.27,183.2,400.5,0.00
6,CT,Connecticut,state,259.62,1.1,0.1,6.9,9.49,4.2,8.9,13.10,4.3,6.9,11.16,0.0,0.0,0.00
7,DE,Delaware,state,282.19,0.4,0.6,114.7,2.30,0.5,1.0,1.53,7.6,12.4,20.03,26.9,22.9,0.00
8,FL,Florida,state,3764.09,42.6,0.9,56.9,66.31,438.2,933.1,1371.36,171.9,279.0,450.86,3.5,1.8,78.24
9,GA,Georgia,state,2860.84,31.0,18.9,630.4,38.38,74.6,158.9,233.51,59.0,95.8,154.77,57.8,65.4,1154.07


In [22]:
ourRightData = ourStates.intersection(correctStates)
len(ourRightData)


25

In [24]:
business = businessDF[businessDF["state"].isin(ourRightData)]
business

,business_id,name,city,state,category1,category2,category3
0,1SWheh84yJXfytovILXOAQ,Arizona Biltmore Golf Club,Phoenix,AZ,Golf,Active Life,None
2,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,Charlotte,NC,Sushi Bars,Restaurants,Japanese
3,xvX2CttrVhyG2z1dFg_0xw,Farmers Insurance - Paul Lorenz,Goodyear,AZ,Insurance,Financial Services,None
4,HhyxOkGAM07SRYtlQ4wMFQ,Queen City Plumbing,Charlotte,NC,Plumbing,Shopping,Local Services
7,gbQN7vr_caG_A1ugSmGhWg,Supercuts,Las Vegas,NV,Hair Salons,Hair Stylists,Barbers
...,...,...,...,...,...,...,...
192603,p1eUTUB_5hf1enoKYBV3DA,Ken Stewart's Tre Belle,Bath,OH,Restaurants,Italian,None
192604,nqb4kWcOwp8bFxzfvaDpZQ,Sanderson Plumbing,North Las Vegas,NV,Water Purification Services,Water Heater Installation/Repair,Home Services
192606,MiEyUDKTjeci5TMfxVZPpg,Phoenix Pavers,Phoenix,AZ,Home Services,Contractors,Landscaping
192607,zNMupayB2jEHVDOji8sxoQ,Beasley's Barber Shop,Mesa,AZ,Beauty & Spas,Barbers,None


In [27]:
data = ratingsDF.join(business.set_index('business_id'), on='business_id')
data = data.dropna()
data

,review_id,user_id,business_id,stars,date,name,city,state,category1,category2,category3
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,2013-05-07 04:34:36,MountainView Hospital,Las Vegas,NV,Fitness & Instruction,Doctors,Health & Medical
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,2017-01-14 21:30:33,Kelly Cardenas Salon - Hard Rock Hotel & Casino,Las Vegas,NV,Beauty & Spas,Hair Stylists,Makeup Artists
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,2016-11-09 20:09:03,J. Philipp Centers for Family and Cosmetic Den...,Chandler,AZ,Health & Medical,Cosmetic Dentists,Orthodontists
5,fdiNeiN_hoCxCMy2wTRW9g,w31MKYsNFMrjhWxxAb5wIw,eU_713ec6fTGNO4BegRaww,4.0,2013-01-20 13:25:59,La Tavola Italiana,Pittsburgh,PA,Restaurants,Italian,Pizza
7,8e9HxxLjjqc9ez5ezzN7iQ,d6xvYpyzcfbF_AZ8vMB7QA,zvO-PJCpNk4fgAVUnExYAA,1.0,2010-10-05 19:12:35,FOX Sports Grill,Scottsdale,AZ,Sports Bars,Bars,Restaurants
...,...,...,...,...,...,...,...,...,...,...,...
6685894,-zfXs0DOn8VjMAt4pf5zJw,HoipOhcXmPExlG8s2OspMg,7sb2FYLS2sejZKxRYF9mtg,5.0,2017-03-23 22:27:46,Sakana,Las Vegas,NV,Buffets,Sushi Bars,Japanese
6685896,jPCXuSBbI_cPocgoNBhdkg,V6BjjQICDO4q7TT3ZhaPWw,yA6dKNm_zl1ucZCnwW8ZCg,1.0,2013-05-22 15:32:08,Italian Grotto,Scottsdale,AZ,Local Flavor,Restaurants,Steakhouses
6685897,zwzO3yPdGbdgGy3-XQ0SWg,PFiIECX8wuvi7P-1mCvIjA,a192hdM0_UVCYLwPJv1Qwg,5.0,2018-11-13 21:15:20,Casa Don Juan - Downtown,Las Vegas,NV,Restaurants,Mexican,Seafood
6685898,o8Nc2BJhKJXM6tRHJDRuPA,wu5R5_N7q5iqCBh6NBX26Q,kOo4ZY2UQAX4j312mzQ8mA,5.0,2018-08-13 18:09:16,Taco Naco,Las Vegas,NV,Mexican,Event Planning & Services,Food Stands


In [38]:
rel = data[['state', 'stars']]
rel = rel.groupby('state').agg(['count', 'mean'])
rel['state'] = rel.index
rel.columns

MultiIndex([('stars', 'count'),
            ('stars',  'mean'),
            ('state',      '')],
           )

In [41]:
fig = go.Figure(data=go.Choropleth(
    locations=rel['state'], # Spatial coordinates
    z = rel[('stars', 'mean')].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Reds',
    colorbar_title = "Average Rating",
))

fig.update_layout(
    title_text = '2011 US Agriculture Exports by State',
    geo_scope='usa', # limite map scope to USA
)

fig.show()